<a id='report-mutable-scopes'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Mutable Scopes</span> </p>

This notebook contains the set up of mutable scopes used to achieve high interactivity with reports.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

In [0]:
#!import "ReportScopes"

# Form Entity Scopes

## Helper Scopes

In [0]:
public interface MutableScopeWithWorkspace : IMutableScopeWithStorage<ReportStorage> {
    protected IWorkspace workspace => GetStorage().Workspace;   
}

## Currency Type

In [0]:
public interface CurrencyFormsEntity : IMutableScope {
    [DropdownEnum(typeof(CurrencyType))]
    CurrencyType CurrencyType { get; set; } 
}

## Scenario

In [0]:
public interface ScenarioFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetScenarioAutocompleteAsync))]
    [Display(Name = "Scenario")]
    string ScenarioControl { get; set; }

    [NotVisible] IDictionary<string, string> ScenarioMapping { get; set; }
    protected string Scenario => !string.IsNullOrWhiteSpace(ScenarioControl) && ScenarioMapping is not null && ScenarioMapping.TryGetValue(ScenarioControl, out var value) ? value : null;
    
    async Task<IReadOnlyCollection<string>> GetScenarioAutocompleteAsync(string userInput, int page, int pageSize) {
        (ScenarioMapping, var orderedDropDownValues) = await workspace.GetAutocompleteMappings<Scenario>(true);
        return orderedDropDownValues.Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).ToArray(); 
    } 
}

## Reporting Node

In [0]:
[InitializeScope(nameof(InitReportingNode))]
public interface ReportingNodeFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetReportingNodeAutocompleteAsync))]
    [Display(Name = "ReportingNode")]
    string ReportingNodeControl { get; set; }

    [NotVisible] IDictionary<string, string>  ReportingNodeMapping { get; set; }
    protected string ReportingNode => !string.IsNullOrWhiteSpace(ReportingNodeControl) && ReportingNodeMapping is not null && ReportingNodeMapping.TryGetValue( ReportingNodeControl, out var value)
        ? value
        : GetStorage().InitialReportingNode.SystemName; // Maybe these cases can be more specific

    async Task<IReadOnlyCollection<string>> GetReportingNodeAutocompleteAsync(string userInput, int page, int pageSize) {
        (ReportingNodeMapping, var orderedDropDownValues) = await workspace.GetAutocompleteMappings<ReportingNode>();
        return orderedDropDownValues.Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).ToArray(); 
    }

    async void InitReportingNode() {
        ReportingNodeControl = GetStorage().InitialReportingNode.DisplayName;
    }
}

## Reporting Period

### Monthly Period

In [0]:
[InitializeScope(nameof(InitReportingPeriod))]
public interface MonthlyPeriodFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetReportingPeriodAutocompleteAsync))]
    string ReportingPeriod { get; set; }

    private char separator => 'M';
    private string[] ReportingPeriodSplit => ReportingPeriod.Split(separator);
    private int ParseReportingPeriod(int index) => !string.IsNullOrWhiteSpace(ReportingPeriod) && ReportingPeriodSplit is not null && Int32.TryParse(ReportingPeriodSplit.ElementAtOrDefault(index), out int value)
        ? value
        : default;

    protected int Year => ParseReportingPeriod(0);
    protected int Month => ParseReportingPeriod(1);

    async Task<IReadOnlyCollection<string>> GetReportingPeriodAutocompleteAsync(string userInput, int page, int pageSize) => 
        await workspace.Query<PartitionByReportingNodeAndPeriod>()
            .Where(x => x.Scenario == null)
            .OrderByDescending(x => x.Year)
            .ThenByDescending(x => x.Month)
            .Select(x => ParseReportingPeriodToDisplayString(x.Year, x.Month, separator))
            .Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase))
            .ToArrayAsync();

    async void InitReportingPeriod() {
        ReportingPeriod = ParseReportingPeriodToDisplayString(GetStorage().InitialReportingPeriod.Year, GetStorage().InitialReportingPeriod.Month, separator);
    }
}

## Filters

In [0]:
public interface BasicFilterFormsEntity : MutableScopeWithWorkspace {
    [DropdownValues("", "GroupOfContract", "Novelty", "EconomicBasis")]
    string FilterName { get; set; }

    [DropdownMethod(nameof(GetBasicFilterAsync))]
    string FilterValue { get; set; }

    [DropdownValues("", "Add", "Remove")]
    string FilterAction { get; set; }
    
    async Task<IReadOnlyCollection<string>> GetBasicFilterAsync(string userInput, int page, int pageSize) =>
        new string[]{ null }.Concat(
            await (FilterName switch
            {
                "GroupOfContract" => workspace.Query<DataNode>().Select(x => x.SystemName),
                "Novelty" => workspace.Query<Novelty>().Select(x => x.SystemName),
                "EconomicBasis" => workspace.Query<EconomicBasis>().Select(x => x.SystemName),
                _ => Enumerable.Empty<string>().AsQueryable()
            }) .Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).OrderBy(x => x).ToArrayAsync()).ToArray();
        
    [NotVisible] IReadOnlyCollection<(string filterName, string filterValue)> InputDataFilter { get; set; }
    
    // This is just a cast... do we need this?
    protected (string filterName, object filterValue)[] dataFilter => (InputDataFilter is null ? Enumerable.Empty<(string, object)>() : InputDataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();

    IReadOnlyCollection<(string filterName, string filterValue)> GetFilters()
    {
        if(FilterAction == "Add")
            AddFilter(FilterName, FilterValue);
        if(FilterAction == "Remove")
            RemoveFilter(FilterName, FilterValue);
        return InputDataFilter;
    }

    private void AddFilter(string filterName, string filterValue)
    {
        if(InputDataFilter == null)
            InputDataFilter = Enumerable.Empty<(string, string)>().ToArray();
        if(!InputDataFilter.Contains((filterName, filterValue)))
            InputDataFilter = InputDataFilter.Append((filterName, filterValue)).ToArray();
    }
    
    private void RemoveFilter(string filterName, string filterValue)
    {
        if(InputDataFilter != null && InputDataFilter.Contains((filterName, filterValue)))
        {   var f = InputDataFilter.ToList();
            f.Remove((filterName, filterValue));
            InputDataFilter = f.ToArray();
        }
    }
}

## Slice and Dice

In [0]:
public interface BasicSliceAndDiceFormsEntity : MutableScopeWithWorkspace {
    [DropdownValues("", "GroupOfContract")]
    string SliceRowName { get; set; }

    [DropdownMethod(nameof(GetSliceColumnNameAutocomplete))]
    string SliceColumnName { get; set; }

    protected IReadOnlyCollection<string> InputRowSlices => (SliceRowName is null ? Enumerable.Empty<string>() : SliceRowName.RepeatOnce()).ToArray();
    [NotVisible] IReadOnlyCollection<string> defaultRowSlices { get; set; }
    protected string[] rowSlices => defaultRowSlices.Union(InputRowSlices).ToArray();

    protected IReadOnlyCollection<string> InputColumnSlices => (SliceColumnName is null ? Enumerable.Empty<string>() : SliceColumnName.RepeatOnce()).ToArray();
    [NotVisible] IReadOnlyCollection<string> defaultColumnSlices { get; set; }
    protected string[] columnSlices => defaultColumnSlices.Union(InputColumnSlices).ToArray();

    IReadOnlyCollection<string> GetSliceColumnNameAutocomplete() => new [] {"", "GroupOfContract", "AmountType"};
}

# Report Scopes

In [0]:
[InitializeScope(nameof(InitReportStorageScopeAsync))]
public interface ReportScope : IMutableScope<string>, ReportingNodeFormsEntity, MonthlyPeriodFormsEntity, ScenarioFormsEntity, CurrencyFormsEntity, BasicSliceAndDiceFormsEntity, BasicFilterFormsEntity {
    protected Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report => GetStorage().Report;
    protected int headerColumnWidthValue => 250;

    HashSet<(ReportIdentity, CurrencyType)> GetDataIdentities() => GetStorage().GetIdentities((Year, Month), ReportingNode, Scenario, CurrencyType); // TODO, add filter for identities, if the property is exposed at this level
    IDataCube<ReportVariable> GetDataCube(IDataCube<ReportVariable> data) => InputDataFilter == null ? data : data.Filter(dataFilter);
       
    async Task<GridOptions> GetReportTaskAsync(IDataCube<ReportVariable> data) {
        return await report.ForDataCube(data)
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions(headerColumnWidth: headerColumnWidthValue)
            .ExecuteAsync();
    }

    async Task InitReportStorageScopeAsync() { // This has the Async issue, but imo it should come in the future
       await GetStorage().InitializeReportIndependentCacheAsync();
    }
}

In [0]:
public interface Data : IMutableScope<((int year, int month) reportingPeriod, string reportingNode, string scenario, CurrencyType currencyType, (string filterName, object filterValue)[] dataFilter)> {
    IDataCube<ReportVariable> DataCube { get; set; }
    
    IDataCube<ReportVariable> Cube { get {
        if(DataCube is null) return Enumerable.Empty<ReportVariable>().ToDataCube();

        if(Identity.scenario != "Delta") return DataCube;
        var bestEstimateById = DataCube.Where(x => x.Scenario == null).ToDictionary(x => x.ToIdentityString());
        return DataCube.Select(x => x.Scenario == null ? x : x with { Value = x.Value - (bestEstimateById.TryGetValue((x with {Scenario = null}).ToIdentityString(), out var be)? be.Value : 0d) }).ToDataCube();
    }}
} 

# Report Scopes

## Best Estimate PV

In [0]:
[InitializeScope(nameof(Init))]
public interface PvReport : ReportScope {

    public IDataCube<ReportVariable> GetData() => GetScopes<LockedBestEstimate>(GetDataIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetDataIdentities()).Aggregate().CurrentBestEstimate;

    async Task<GridOptions> ToReportAsync() {
        await GetStorage().InitializeAsync((Year, Month), ReportingNode, Scenario, CurrencyType);
        var dataScope = GetScope<Data>(((Year, Month), ReportingNode, Scenario, CurrencyType, dataFilter));
        dataScope.DataCube = GetData();
        return await GetReportTaskAsync(dataScope.DataCube);
        return await GetReportTaskAsync(GetDataCube(GetData()));
    }

     void Init() {
         // BasicSliceAndDiceFormsEntity
         defaultRowSlices = new string[] { "Novelty", "VariableType" };
         defaultColumnSlices = new string[] { "Currency", "LiabilityType", "EconomicBasis" };
    }
}

## Risk Adjustment PV

# Mutable Scope

The IIfrs17Report mutable scope is created with applicabilities to control how the data for each individual report is retrieved from the [report scopes](./ReportScopes)

In [0]:
public interface ReportUniverse : IMutableScopeWithStorage<ReportStorage>{}

In [0]:
[InitializeScope(nameof(InitAsync))]
public interface IIfrs17Report : IMutableScope<string, ReportStorage> {
    // Infrastructure
    protected IWorkspace workspace => GetStorage().Workspace;
    protected Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report => GetStorage().Report;

    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<IIfrs17Report>(s => s.WithApplicability<PvReport>(x => x.Identity == nameof(PvReport))
                                              .WithApplicability<RaReport>(x => x.Identity == nameof(RaReport))
                                              .WithApplicability<WrittenReport>(x => x.Identity == nameof(WrittenReport))
                                              .WithApplicability<AccrualReport>(x => x.Identity == nameof(AccrualReport))
                                              .WithApplicability<DeferralReport>(x => x.Identity == nameof(DeferralReport))
                                              .WithApplicability<FcfReport>(x => x.Identity == nameof(FcfReport))
                                              .WithApplicability<ExpAdjReport>(x => x.Identity == nameof(ExpAdjReport))
                                              .WithApplicability<TmReport>(x => x.Identity == nameof(TmReport))
                                              .WithApplicability<CsmReport>(x => x.Identity == nameof(CsmReport))
                                              .WithApplicability<ActLrcReport>(x => x.Identity == nameof(ActLrcReport))
                                              .WithApplicability<LrcReport>(x => x.Identity == nameof(LrcReport))
                                              .WithApplicability<ActLicReport>(x => x.Identity == nameof(ActLicReport))
                                              .WithApplicability<LicReport>(x => x.Identity == nameof(LicReport))
                                              .WithApplicability<FpReport>(x => x.Identity == nameof(FpReport))
                );

     // Basic mutable properties
    (int Year, int Month) ReportingPeriod { get; set; }
    int Year => ReportingPeriod.Year;
    int Month => ReportingPeriod.Month;
    string ReportingNode { get; set; }
    string Scenario { get; set; }  // TODO: Enable dropdown selection including All and Delta
    string Comparison { get; set; }  // TODO: only for scenario at the beginning, meant to enable general purpose comparisons 
    CurrencyType CurrencyType { get; set; }
    
    ((int Year, int Month) ReportingPeriod, string ReportingNode, string Scenario, CurrencyType) ShowSettings => (ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Slice and Dice
    protected string[] forbiddenSlices => new string[] { };

    IEnumerable<string> RowSlices { get; set; }
    protected string[] defaultRowSlices => new string[] { };
    protected string[] rowSlices => RowSlices is null ? defaultRowSlices : defaultRowSlices.Concat(RowSlices).Where(x => !forbiddenSlices.Contains(x)).ToArray();

    IEnumerable<string> ColumnSlices { get; set; }
    protected string[] defaultColumnSlices => new string[] { };
    protected string[] columnSlices { get{
        var slices = ColumnSlices is null ? defaultColumnSlices : defaultColumnSlices.Where(cs => !ColumnSlices.Contains(cs)).Concat(ColumnSlices).Where(x => !forbiddenSlices.Contains(x)).ToArray();
        return Scenario == "All" || Scenario == "Delta"
            ? slices.Concat(nameof(Scenario).RepeatOnce()).ToArray() 
            : Scenario is null ? slices : nameof(Scenario).RepeatOnce().Concat(slices).ToArray();
    }}

    // Identities
    protected HashSet<(ReportIdentity, CurrencyType)> GetIdentities() => GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Filter
    IEnumerable<(string filterName, string filterValue)> DataFilter { get; set; }
    protected (string filterName, object filterValue)[] dataFilter => (DataFilter is null ? Enumerable.Empty<(string, object)>() : DataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();

    // Scope Initialization
    async Task InitAsync() {
        var mostRecentPartition = (await workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();  
        ReportingPeriod = (mostRecentPartition.Year, mostRecentPartition.Month);
        ReportingNode = (await workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName; // TODO: change once user permissions are available
        Scenario = null;
        CurrencyType = CurrencyType.Contractual;
        await GetStorage().InitializeReportIndependentCacheAsync();
    }  

    //Report
    public IDataCube<ReportVariable> GetDataCube() => default;
    protected int headerColumnWidthValue => 250;
    public Task<GridOptions> ToReportAsync => GetReportTaskAsync();
    private async Task<GridOptions> GetReportTaskAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);   
        return await report.ForDataCube(GetScope<Data>((ReportingPeriod, ReportingNode, Scenario, CurrencyType, Identity, dataFilter)).Cube)
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions(headerColumnWidth: headerColumnWidthValue)
            .ExecuteAsync();
    }
}

public interface PvReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty", "VariableType" };
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? (GetScopes<LockedBestEstimate>(GetIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetIdentities()).Aggregate().CurrentBestEstimate)
                                                                : (GetScopes<LockedBestEstimate>(GetIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetIdentities()).Aggregate().CurrentBestEstimate).Filter(dataFilter);
}

public interface RaReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty", "VariableType" };
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LockedRiskAdjustment>(GetIdentities()).Aggregate().LockedRiskAdjustment + 
                                                             GetScopes<CurrentRiskAdjustment>(GetIdentities()).Aggregate().CurrentRiskAdjustment
                                                                : GetScopes<LockedRiskAdjustment>(GetIdentities()).Aggregate().LockedRiskAdjustment.Filter(dataFilter) + 
                                                             GetScopes<CurrentRiskAdjustment>(GetIdentities()).Aggregate().CurrentRiskAdjustment.Filter(dataFilter);
}

public interface WrittenReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {nameof(EconomicBasis)};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "LiabilityType"};
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Written
                                                                : GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Written.Filter(dataFilter);
}

public interface AccrualReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType"};
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Advance + 
                                                             GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Overdue
                                                                : GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Advance.Filter(dataFilter) + 
                                                             GetScopes<WrittenAndAccruals>(GetIdentities()).Aggregate().Overdue.Filter(dataFilter);
}

public interface DeferralReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "LiabilityType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Deferrals>(GetIdentities()).Aggregate().Deferrals
                                                                : GetScopes<Deferrals>(GetIdentities()).Aggregate().Deferrals.Filter(dataFilter);
}

public interface FcfReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty","VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "LiabilityType", "EconomicBasis" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() =>  DataFilter == null 
                                                                ? GetScopes<Fcf>(GetIdentities()).Aggregate().Fcf
                                                                : GetScopes<Fcf>(GetIdentities()).Aggregate().Fcf.Filter(dataFilter);
}

public interface ExpAdjReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {"EstimateType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "AmountType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<ExperienceAdjustment>(GetIdentities()).Aggregate().ActuarialExperienceAdjustment
                                                                : GetScopes<ExperienceAdjustment>(GetIdentities()).Aggregate().ActuarialExperienceAdjustment.Filter(dataFilter);
}
public interface TmReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType", nameof(EconomicBasis)};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty", "VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LrcTechnicalMargin>(GetIdentities()).Aggregate().LrcTechnicalMargin
                                                                : GetScopes<LrcTechnicalMargin>(GetIdentities()).Aggregate().LrcTechnicalMargin.Filter(dataFilter);
}
public interface CsmReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType", nameof(EconomicBasis)};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty", "VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Csm>(GetIdentities()).Aggregate().Csm + 
                                                             GetScopes<Lc>(GetIdentities()).Aggregate().Lc + 
                                                             GetScopes<Loreco>(GetIdentities()).Aggregate().Loreco
                                                                : GetScopes<Csm>(GetIdentities()).Aggregate().Csm.Filter(dataFilter) + 
                                                             GetScopes<Lc>(GetIdentities()).Aggregate().Lc.Filter(dataFilter) + 
                                                             GetScopes<Loreco>(GetIdentities()).Aggregate().Loreco.Filter(dataFilter);
}
public interface ActLrcReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty","VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LrcActuarial>(GetIdentities()).Aggregate().LrcActuarial
                                                                : GetScopes<LrcActuarial>(GetIdentities()).Aggregate().LrcActuarial.Filter(dataFilter);
}
public interface LrcReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Lrc>(GetIdentities()).Aggregate().Lrc
                                                                : GetScopes<Lrc>(GetIdentities()).Aggregate().Lrc.Filter(dataFilter);
}
public interface ActLicReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"Novelty","VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<LicActuarial>(GetIdentities()).Aggregate().LicActuarial
                                                                : GetScopes<LicActuarial>(GetIdentities()).Aggregate().LicActuarial.Filter(dataFilter);
}
public interface LicReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType"};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency", "EstimateType" };
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<Lic>(GetIdentities()).Aggregate().Lic
                                                                : GetScopes<Lic>(GetIdentities()).Aggregate().Lic.Filter(dataFilter);
}
public interface FpReport : IIfrs17Report {
    string[] IIfrs17Report.forbiddenSlices => new string[] {"AmountType", nameof(EconomicBasis)};
    string[] IIfrs17Report.defaultRowSlices => new string[] {"VariableType", "EstimateType"};
    string[] IIfrs17Report.defaultColumnSlices => new string[] { "Currency","LiabilityType" };
    int IIfrs17Report.headerColumnWidthValue => 500;
    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => DataFilter == null 
                                                                ? GetScopes<FinancialPerformance>(GetIdentities()).Aggregate().FinancialPerformance
                                                                : GetScopes<FinancialPerformance>(GetIdentities()).Aggregate().FinancialPerformance.Filter(dataFilter);
}

public interface Data : IMutableScope<((int year, int month) period, string reportingNode, string scenario, CurrencyType currencyType, 
                                        string reportType, (string filterName, object filterValue)[] dataFilter)> 
{
    IDataCube<ReportVariable> Cube { get{
        var dataCube = GetScope<IIfrs17Report>(Identity.reportType).GetDataCube();
        // TODO: suggestion to place the filter here instead of having it in every applicability scope
        if(Identity.scenario != "Delta") return dataCube;
        var bestEstimateById = dataCube.Where(x => x.Scenario == null).ToDictionary(x => x.ToIdentityString());
        return dataCube.Select(x => x.Scenario == null ? x : x with { Value = x.Value - (bestEstimateById.TryGetValue((x with {Scenario = null}).ToIdentityString(), out var be)? be.Value : 0.0) }).ToDataCube();
    }}
} 

# IFRS 17 Reports

This class is used to trigger the calculation of the reports and it is exposed to the end-user in the reports.

In [0]:
public class Ifrs17 
{
    private Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes;
    private Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report;
    private ReportStorage Storage;
    private ReportUniverse reportUniverse;
    
    //reset
    public void Reset(IWorkspace workspace) => Storage = new ReportStorage(workspace, report);

    //constructor
    public Ifrs17 (IWorkspace workspace, Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes, Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report)
    {
        this.scopes = scopes; 
        this.report = report; 
        Storage = new ReportStorage(workspace, report);
        reportUniverse = scopes.ForSingleton().WithStorage(Storage).ToScope<ReportUniverse>();
    }

    public IIfrs17Report PresentValues => reportUniverse.GetScope<PvReport>();
    public IIfrs17Report RiskAdjustments => reportUniverse.GetScope<RaReport>();
    public IIfrs17Report WrittenActuals => reportUniverse.GetScope<WrittenReport>();
    public IIfrs17Report AccrualActuals => reportUniverse.GetScope<AccrualReport>();
    public IIfrs17Report DeferralActuals => reportUniverse.GetScope<DeferralReport>();
    public IIfrs17Report FulfillmentCashflows => reportUniverse.GetScope<FcfReport>();
    public IIfrs17Report ExperienceAdjustments => reportUniverse.GetScope<ExpAdjReport>();
    public IIfrs17Report TechnicalMargins => reportUniverse.GetScope<TmReport>();
    public IIfrs17Report AllocatedTechnicalMargins => reportUniverse.GetScope<CsmReport>();
    public IIfrs17Report ActuarialLrc => reportUniverse.GetScope<ActLrcReport>();
    public IIfrs17Report Lrc => reportUniverse.GetScope<LrcReport>();
    public IIfrs17Report ActuarialLic => reportUniverse.GetScope<ActLicReport>();
    public IIfrs17Report Lic => reportUniverse.GetScope<LicReport>();
    public IIfrs17Report FinancialPerformance => reportUniverse.GetScope<FpReport>();
}